# Imports

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Data preparation


In [2]:
movies = pd.read_csv('datasets/movies.csv').drop('genres', axis=1)
ratings = pd.read_csv('datasets/ratings.csv')

In [3]:
movies = pd.read_csv('datasets/movies.csv').drop('genres', axis=1)
ratings = pd.read_csv('datasets/ratings.csv')

ratings = ratings.join(movies, on='movieId', how='left', lsuffix="_m")

ratings.drop(['movieId', 'movieId_m', 'timestamp'], axis=1, inplace=True)

positive = ratings.drop(ratings[ratings.rating < 4].index,
                        axis=0,
                        inplace=False).drop('rating',
                                           axis=1)

negative = ratings.drop(ratings[ratings.rating > 2].index,
                        axis=0, 
                        inplace=False).drop('rating',
                                            axis=1)
positive = positive.pivot_table(index='userId', 
                     columns='title', 
                     values='title', 
                     aggfunc=lambda x: ' '.join(x)).apply(lambda x: x==x)

negative = negative.pivot_table(index='userId', 
                     columns='title', 
                     values='title', 
                     aggfunc=lambda x: ' '.join(x)).apply(lambda x: x==x)

# Association rules

In [4]:
frequent_itemset_positive = apriori(positive, min_support=0.05, use_colnames=True)
positive_rules = association_rules(frequent_itemset_positive, metric="confidence", min_threshold=0.5)

frequent_itemset_negative = apriori(negative, min_support=0.05, use_colnames=True)
negative_rules = association_rules(frequent_itemset_positive, metric="confidence", min_threshold=0.5)

# Predictions

## Movies recommended to you
Provide movies which you like below

In [5]:
preferences = {'Inkheart (2008)', 'Daytrippers, The (1996)'}

In [6]:
idx =  positive_rules['antecedents'].apply(lambda x: x.issuperset(preferences))
set([list(x)[0] for x in list(positive_rules[idx].consequents.drop_duplicates())])

{'Angus (1995)',
 'Bird on a Wire (1990)',
 'Crow, The (1994)',
 'Departed, The (2006)',
 'Desperate Measures (1998)',
 'Front Page, The (1974)',
 'Infernal Affairs 2 (Mou gaan dou II) (2003)',
 'Keys to Tulsa (1997)',
 'Mirage (1995)',
 'Once Were Warriors (1994)',
 'Pal Joey (1957)',
 'Serpent and the Rainbow, The (1988)',
 'Soul Food (1997)',
 'Southpaw (2015)',
 'Tom & Viv (1994)',
 'Van, The (1996)'}

## Movies defienietly not recommended to you
Provide movies which you don't like below

In [7]:
dislikes = {'Soul Food (1997)', 'Keys to Tulsa (1997)'}

In [8]:
idx =  negative_rules['antecedents'].apply(lambda x: x.issuperset(dislikes))
set([list(x)[0] for x in list(negative_rules[idx].consequents)])

{'Angus (1995)',
 'Bambi (1942)',
 'Battle for the Planet of the Apes (1973)',
 'Beauty and the Beast (1991)',
 'Bird on a Wire (1990)',
 'Boomerang (1992)',
 'Crash (1996)',
 'Crow, The (1994)',
 'Daytrippers, The (1996)',
 'Desperate Measures (1998)',
 'Fantasia (1940)',
 'Front Page, The (1974)',
 'Hidden, The (1987)',
 'In Love and War (1996)',
 'Infernal Affairs 2 (Mou gaan dou II) (2003)',
 'Inkheart (2008)',
 'Jumanji (1995)',
 'Mirage (1995)',
 'Once Were Warriors (1994)',
 'Pal Joey (1957)',
 'Pillow Book, The (1996)',
 'Simple Twist of Fate, A (1994)',
 'Superman II (1980)',
 'Swingers (1996)',
 'Tom & Viv (1994)',
 'True Crime (1996)',
 'Van, The (1996)'}